# Ensemble Model

In [1]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import OneHotEncoder
from sklearn.dummy import DummyRegressor
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_squared_error
from typing import Union

from raw_to_transformed_data import get_sql_data

np.set_printoptions(suppress=True)

In [2]:
ModelRegressor = Union[
    LinearRegression, RandomForestRegressor, GradientBoostingRegressor]

In [3]:
def cv_regression_model(
        model: ModelRegressor, X: pd.DataFrame, y: pd.DataFrame, 
        scoring: str="neg_mean_squared_error", 
        cv: int=5) -> np.ndarray:
    """Perform cross validation on regression models."""
    rmses = cross_val_score(
        model, X, y, scoring=scoring, cv=cv)
    avg_rmse = np.mean(np.sqrt(-rmses))
    return avg_rmse, np.sqrt(-rmses)

In [4]:
# Get crashes data
query_crashes = """
SELECT *
FROM crashes_joined;
"""
dbname = "chi-traffic-accidents"
df_crashes = get_sql_data(dbname, query_crashes)

In [5]:
# limiting to obs. with >0 injured
# df_crashes = df_crashes.loc[df_crashes["has_injuries"]==1, :]

In [6]:
df_crashes.head()

,crash_record_id,crash_date,posted_speed_limit,traffic_control_device,device_condition,weather_condition,lighting_condition,first_crash_type,trafficway_type,alignment,roadway_surface_cond,road_defect,report_type,prim_contributory_cause,street_direction,num_units,injuries_total,crash_hour,crash_day_of_week,crash_month,intersection_related_i,hit_and_run_i,lane_cnt,has_injuries,num_bikes_involved,num_pedestrians_involved,num_partially_ejected,num_extricated
0,7a63945a9bae12c05dee82c1962bf40f23cdd02ca655d7...,2021-08-01 03:42:00,30,NO CONTROLS,NO CONTROLS,CLEAR,"DARKNESS, LIGHTED ROAD",FIXED OBJECT,NOT DIVIDED,STRAIGHT AND LEVEL,DRY,UNKNOWN,ON SCENE,DISREGARDING OTHER TRAFFIC SIGNS,S,2,0.0,3,Sunday,August,None,None,NaN,0,0.0,0.0,0.0,0.0
1,d207ce72f4348fa06dddcd98628d0dec81d4f195cb8d8b...,2021-08-01 02:41:00,35,TRAFFIC SIGNAL,FUNCTIONING PROPERLY,CLEAR,"DARKNESS, LIGHTED ROAD",ANGLE,FOUR WAY,STRAIGHT AND LEVEL,DRY,NO DEFECTS,NOT ON SCENE (DESK REPORT),DISREGARDING TRAFFIC SIGNALS,N,2,1.0,2,Sunday,August,Y,Y,NaN,1,0.0,0.0,0.0,0.0
2,bba4db7cfef29230aa5d24f6b8912a2539b5d6f481a94e...,2021-08-01 02:26:00,25,NO CONTROLS,NO CONTROLS,CLEAR,"DARKNESS, LIGHTED ROAD",SIDESWIPE SAME DIRECTION,NOT DIVIDED,STRAIGHT AND LEVEL,DRY,UNKNOWN,ON SCENE,IMPROPER TURNING/NO SIGNAL,S,2,0.0,2,Sunday,August,None,None,NaN,0,0.0,0.0,0.0,0.0
3,1e39110c6e01fd2f24e282cd610eb90d6f98a6241ab193...,2021-08-01 02:22:00,30,TRAFFIC SIGNAL,FUNCTIONING PROPERLY,CLEAR,"DARKNESS, LIGHTED ROAD",TURNING,OTHER,STRAIGHT AND LEVEL,DRY,NO DEFECTS,ON SCENE,UNABLE TO DETERMINE,S,2,1.0,2,Sunday,August,N,Y,NaN,1,0.0,0.0,0.0,0.0
4,d20d9db8a88fc227d48392b45380ae57898e5fcdcda7db...,2021-08-01 02:08:00,30,NO CONTROLS,NO CONTROLS,CLEAR,DUSK,PARKED MOTOR VEHICLE,ONE-WAY,STRAIGHT AND LEVEL,DRY,NO DEFECTS,ON SCENE,IMPROPER LANE USAGE,W,3,2.0,2,Sunday,August,None,Y,NaN,1,0.0,0.0,0.0,0.0


In [7]:
df_crashes.shape

(526569, 28)

In [8]:
drop_additional = True

In [9]:
# Transforming df_crashes for preliminary model
drop_cols = ['crash_record_id', 'crash_date', 'report_type', 
    'prim_contributory_cause', 'intersection_related_i', 'hit_and_run_i', 
    'lane_cnt', "has_injuries"]

if drop_additional:
    drop_additional = ['num_bikes_involved', 'num_extricated', 'num_partially_ejected', 
        'num_pedestrians_involved']
else:
    drop_additional = []
    
df_crashes = df_crashes.drop(columns=drop_cols+drop_additional)

df_crashes = df_crashes.rename(columns={"crash_day_of_week": "crash_day"})
df_crashes["street_direction"] = (
    df_crashes["street_direction"]
        .fillna(df_crashes["street_direction"].mode()[0]))

In [10]:
df_crashes.head()

,posted_speed_limit,traffic_control_device,device_condition,weather_condition,lighting_condition,first_crash_type,trafficway_type,alignment,roadway_surface_cond,road_defect,street_direction,num_units,injuries_total,crash_hour,crash_day,crash_month
0,30,NO CONTROLS,NO CONTROLS,CLEAR,"DARKNESS, LIGHTED ROAD",FIXED OBJECT,NOT DIVIDED,STRAIGHT AND LEVEL,DRY,UNKNOWN,S,2,0.0,3,Sunday,August
1,35,TRAFFIC SIGNAL,FUNCTIONING PROPERLY,CLEAR,"DARKNESS, LIGHTED ROAD",ANGLE,FOUR WAY,STRAIGHT AND LEVEL,DRY,NO DEFECTS,N,2,1.0,2,Sunday,August
2,25,NO CONTROLS,NO CONTROLS,CLEAR,"DARKNESS, LIGHTED ROAD",SIDESWIPE SAME DIRECTION,NOT DIVIDED,STRAIGHT AND LEVEL,DRY,UNKNOWN,S,2,0.0,2,Sunday,August
3,30,TRAFFIC SIGNAL,FUNCTIONING PROPERLY,CLEAR,"DARKNESS, LIGHTED ROAD",TURNING,OTHER,STRAIGHT AND LEVEL,DRY,NO DEFECTS,S,2,1.0,2,Sunday,August
4,30,NO CONTROLS,NO CONTROLS,CLEAR,DUSK,PARKED MOTOR VEHICLE,ONE-WAY,STRAIGHT AND LEVEL,DRY,NO DEFECTS,W,3,2.0,2,Sunday,August


In [11]:
df_crashes.shape

(526569, 16)

In [12]:
# Create X and y
y = df_crashes.pop("injuries_total")
X = df_crashes.copy()
# del df_crashes

In [13]:
y.shape

(526569,)

In [14]:
X.shape

(526569, 15)

In [15]:
X.head()

,posted_speed_limit,traffic_control_device,device_condition,weather_condition,lighting_condition,first_crash_type,trafficway_type,alignment,roadway_surface_cond,road_defect,street_direction,num_units,crash_hour,crash_day,crash_month
0,30,NO CONTROLS,NO CONTROLS,CLEAR,"DARKNESS, LIGHTED ROAD",FIXED OBJECT,NOT DIVIDED,STRAIGHT AND LEVEL,DRY,UNKNOWN,S,2,3,Sunday,August
1,35,TRAFFIC SIGNAL,FUNCTIONING PROPERLY,CLEAR,"DARKNESS, LIGHTED ROAD",ANGLE,FOUR WAY,STRAIGHT AND LEVEL,DRY,NO DEFECTS,N,2,2,Sunday,August
2,25,NO CONTROLS,NO CONTROLS,CLEAR,"DARKNESS, LIGHTED ROAD",SIDESWIPE SAME DIRECTION,NOT DIVIDED,STRAIGHT AND LEVEL,DRY,UNKNOWN,S,2,2,Sunday,August
3,30,TRAFFIC SIGNAL,FUNCTIONING PROPERLY,CLEAR,"DARKNESS, LIGHTED ROAD",TURNING,OTHER,STRAIGHT AND LEVEL,DRY,NO DEFECTS,S,2,2,Sunday,August
4,30,NO CONTROLS,NO CONTROLS,CLEAR,DUSK,PARKED MOTOR VEHICLE,ONE-WAY,STRAIGHT AND LEVEL,DRY,NO DEFECTS,W,3,2,Sunday,August


In [16]:
# Transforming X and y for modeling
numeric_cols = ["posted_speed_limit", "num_units", "crash_hour"]
category_cols = X.columns.difference(numeric_cols)

In [17]:
encoder = OneHotEncoder(drop=None, sparse=True)
onehot_crashes = encoder.fit_transform(X[category_cols])

In [18]:
matrix_cols = []
for col, ele in zip(category_cols, encoder.categories_):
    for e in ele:
        matrix_cols.append(col + "_" + e.lower())

In [19]:
# matrix_cols

In [20]:
X[numeric_cols].reset_index()

,index,posted_speed_limit,num_units,crash_hour
0,0,30,2,3
1,1,35,2,2
2,2,25,2,2
3,3,30,2,2
4,4,30,3,2
...,...,...,...,...
526564,526564,30,2,19
526565,526565,30,2,19
526566,526566,30,2,7
526567,526567,30,2,18


In [21]:
pd.DataFrame(onehot_crashes.toarray(), columns=matrix_cols)

,alignment_curve on grade,alignment_curve on hillcrest,"alignment_curve, level",alignment_straight and level,alignment_straight on grade,alignment_straight on hillcrest,crash_day_friday,crash_day_monday,crash_day_saturday,crash_day_sunday,crash_day_thursday,crash_day_tuesday,crash_day_wednesday,crash_month_april,crash_month_august,crash_month_december,crash_month_february,crash_month_january,crash_month_july,crash_month_june,crash_month_march,crash_month_may,crash_month_november,crash_month_october,crash_month_september,device_condition_functioning improperly,device_condition_functioning properly,device_condition_missing,device_condition_no controls,device_condition_not functioning,device_condition_other,device_condition_unknown,device_condition_worn reflective material,first_crash_type_angle,first_crash_type_animal,first_crash_type_fixed object,first_crash_type_head on,first_crash_type_other noncollision,first_crash_type_other object,first_crash_type_overturned,first_crash_type_parked motor vehicle,first_crash_type_pedalcyclist,first_crash_type_pedestrian,first_crash_type_rear end,first_crash_type_rear to front,first_crash_type_rear to rear,first_crash_type_rear to side,first_crash_type_sideswipe opposite direction,first_crash_type_sideswipe same direction,first_crash_type_train,first_crash_type_turning,lighting_condition_darkness,"lighting_condition_darkness, lighted road",lighting_condition_dawn,lighting_condition_daylight,lighting_condition_dusk,lighting_condition_unknown,road_defect_debris on roadway,road_defect_no defects,road_defect_other,"road_defect_rut, holes",road_defect_shoulder defect,road_defect_unknown,road_defect_worn surface,roadway_surface_cond_dry,roadway_surface_cond_ice,roadway_surface_cond_other,"roadway_surface_cond_sand, mud, dirt",roadway_surface_cond_snow or slush,roadway_surface_cond_unknown,roadway_surface_cond_wet,street_direction_e,street_direction_n,street_direction_s,street_direction_w,traffic_control_device_bicycle crossing sign,traffic_control_device_delineators,traffic_control_device_flashing control signal,traffic_control_device_lane use marking,traffic_control_device_no controls,traffic_control_device_no passing,traffic_control_device_other,traffic_control_device_other railroad crossing,traffic_control_device_other reg. sign,traffic_control_device_other warning sign,traffic_control_device_pedestrian crossing sign,traffic_control_device_police/flagman,traffic_control_device_railroad crossing gate,traffic_control_device_rr crossing sign,traffic_control_device_school zone,traffic_control_device_stop sign/flasher,traffic_control_device_traffic signal,traffic_control_device_unknown,traffic_control_device_yield,trafficway_type_alley,trafficway_type_center turn lane,trafficway_type_divided - w/median (not raised),trafficway_type_divided - w/median barrier,trafficway_type_driveway,"trafficway_type_five point, or more",trafficway_type_four way,trafficway_type_l-intersection,trafficway_type_not divided,trafficway_type_not reported,trafficway_type_one-way,trafficway_type_other,trafficway_type_parking lot,trafficway_type_ramp,trafficway_type_roundabout,trafficway_type_t-intersection,trafficway_type_traffic route,trafficway_type_unknown,trafficway_type_unknown intersection type,trafficway_type_y-intersection,"weather_condition_blowing sand, soil, dirt",weather_condition_blowing snow,weather_condition_clear,weather_condition_cloudy/overcast,weather_condition_fog/smoke/haze,weather_condition_freezing rain/drizzle,weather_condition_other,weather_condition_rain,weather_condition_severe cross wind gate,weather_condition_sleet/hail,weather_condition_snow,weather_condition_unknown
0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0

In [22]:



X = pd.concat(
    [X[numeric_cols].reset_index(), pd.DataFrame(
        onehot_crashes.toarray(), columns=matrix_cols)], 
    axis=1)

In [23]:
X.shape

(526569, 130)

In [24]:
y.shape

(526569,)

In [25]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [26]:
# Baseline model
model_dum = DummyRegressor(strategy="mean")
model_dum.fit(X_train, y_train)
y_pred = model_dum.predict(X_test)
rmse_dum = np.sqrt(mean_squared_error(y_test, y_pred))
print(f"RMSE dum: {rmse_dum:.4f}")

RMSE dum: 0.5636


In [27]:
# Evaluate regression models
model_lr = LinearRegression()
model_rf = RandomForestRegressor(
    n_estimators=100,
    max_features="sqrt")
model_gb = GradientBoostingRegressor(
    learning_rate=0.1,
    n_estimators=1000,
    max_depth=3,
    min_samples_leaf=1,
    min_samples_split=2)
models_all = [model_lr, model_rf, model_gb]

In [ ]:
scores = []
score_lists = []
for model in models_all:
    score, lst = cv_regression_model(model, X_train, y_train)
    scores.append(score)
    score_lists.append(lst)
    print(f"RMSE for {model}: {score:.4f}")
    print(lst)

RMSE for LinearRegression(): 0.5129
[0.50614782 0.51748385 0.51795982 0.50963176 0.51309467]
